# Bandgap Workflow Example
 This notebook demonstrates how to build and run a bandgap workflow for a material.

## Process Overview
### 1. Set up the environment and parameters.
### 2. Log in to get the API token
### 3. Load the target material.
### 4. Import workflow builder and related analyzers.
### 5. Analyze material to get parameters for the workflow configuration.
### 6. Create the workflow configuration.
### 7. Create a job with material and workflow configuration.
### 8. Submit the job to the server.
### 9. Monitor the job status and retrieve results.
### 10. Display the results.

## 1. Set up the environment and parameters

## 2. Log in to get the API token

In [ ]:
from mat3ra.api import ApiClient

from examples.reproducing_publications.band_gaps_for_interface_bilayer_twisted_molybdenum_disulfide import \
    unit_flowchart_id

# Log in to get the API token
auth_config = await ApiClient().login()

## 3. Load the target material

In [ ]:
from utils.visualize import visualize_materials as visualize
from utils.jupyterlite import load_material_from_folder

material = load_material_from_folder("/uploads", "MoS2_twisted_interface_60_degrees.json")
visualize(material)

## 4. Import workflow builder and related analyzers

In [ ]:
from mat3ra.wode.context_providers import (
    PointsGridFormDataProvider,  # exists - for k-points
    PlanewaveCutoffsContextProvider,  # exists - for cutoffs
    SmearingContextProvider, # no JS implementation yet
    BandsContextProvider # no JS implementation yet
)


kgrid_provider = PointsGridFormDataProvider(material=material)
cutoffs_provider = PlanewaveCutoffsContextProvider(material=material)
smearing_provider = SmearingContextProvider(material=material)
bands_provider = BandsContextProvider(material=material)

kpoints = kgrid_provider.get_dimensions()  # or calculate_dimensions()
cutoff = cutoffs_provider.get_cutoff()  # defaultECUTWFC, defaultECUTRHO
smearing = smearing_provider.get_smearing()
number_of_bands = bands_provider.get_number_of_bands()

## 5. Create workflow and set its parameters

In [ ]:
from mat3ra.standata.workflows import Workflows
from mat3ra.wode.workflows import Workflow
from mat3ra.wode.pseudopotentials import PseudopotentialEnum

# Get Applications list (with versions, build)
# Search WF by name and and n
workflow_config = Workflows.get_by_name_first_match("band_structure")
workflow = Workflow.create(workflow_config)

# View workflow to understand its structure
visualize(workflow)
# Get workflow's specific unit that needs to be modified
unit_to_modify = workflow.get_unit_by_name(name="scf", subworkflow_name="xxx", order_in_subworkflow=1)
new_context = kgrid_context_provider.get_data(dimentions=[6,6,6])
unit_to_modify.context.add_context(new_context)
workflow.set_unit(unit_flowchart_id=unit_to_modify.flowchart_id, new_unit=unit_to_modify)

workflow.set_context_providers(kpoints)
workflow.set_pseudopotential(PseudopotentialEnum.Funtional_GGA) # elements will be set automatically based on the material

## 6. Create the compute configuration

In [ ]:
from mat3ra.ide.compute import ComputeConfiguration, QueueEnum
compute_config = ComputeConfiguration(
    queue = QueueEnum.OR8,
    nodes = 1,
    cores = 8,
)

## 7. Create the job with material and workflow configuration

In [ ]:
from mat3ra.jode.job import create_job
job = create_job(workflow=workflow, material=material, compute = compute_config, auth_config=auth_config)


## 8. Submit the job and monitor the status

In [ ]:
from mat3ra.prode import PropertyEnum
job.run()
job.wait_for_complete()
# job.check_status()
# job.get_current_output()

## 9. Retrieve results

In [ ]:
# AFTER Finished
# A class from Prode to handle results
results = job.get_results(PropertyEnum.BAND_GAP, PropertyEnum.BAND_STRUCTURE)

## 10. Display results

In [ ]:
# Visual library that can visualize any property defined in Prode
from mat3ra.prove import visualize_property
visualize_property(results.band_structure)
print(results.band_gap)

In [ ]:
import pydot
import json

# Example JSON data (you would load this from a file)
json_data = {
    "nodes": [
        {"name": "A", "shape": "box", "label": "Start"},
        {"name": "B", "shape": "box", "label": "Process 1"},
        {"name": "C", "shape": "diamond", "label": "Decision"},
        {"name": "D", "shape": "box", "label": "End"}
    ],
    "edges": [
        {"source": "A", "target": "B", "label": ""},
        {"source": "B", "target": "C", "label": ""},
        {"source": "C", "target": "D", "label": "Yes"},
        {"source": "C", "target": "B", "label": "No"}
    ]
}

# Create a new directed graph
dot_graph = pydot.Dot("my_flowchart", graph_type="digraph")

# Add nodes
for node_data in json_data["nodes"]:
    node = pydot.Node(node_data["name"], shape=node_data["shape"], label=node_data["label"])
    dot_graph.add_node(node)

# Add edges
for edge_data in json_data["edges"]:
    edge = pydot.Edge(edge_data["source"], edge_data["target"], label=edge_data["label"])
    dot_graph.add_edge(edge)

# Save the flowchart as a PNG
dot_graph.write_png("flowchart.png")
print("Flowchart rendered to flowchart.png")
